# First Project Pipline

## Setup

In [1]:
import spacy
from spacy.tokens import Token
from spacy.tokens import Doc
from spacy.language import Language

In [2]:
from gensim.summarization import keywords

In [3]:
from wordfreq import zipf_frequency
from wordfreq import word_frequency

In [4]:
import numpy as np
import pandas as pd
import regex as re

In [5]:
import nltk
#nltk.download('cmudict')

In [6]:
from nltk.corpus import cmudict

phoneme_dict = dict(cmudict.entries())

def syllable_counter(word):
    '''function that counts a syllable in a word'''
    if word not in phoneme_dict:
        return 0
    syllables = phoneme_dict[word]
    count = len([syllable for syllable in syllables if syllable[-1].isdigit()])
    return count

In [7]:
!python --version

Python 3.8.0


## Data

In [8]:
text_file = '../data/kafka.txt'

In [9]:
#print(open(text_file, "r").read())

# Create Pipeline

### Base Pipeline

In [52]:
nlp = spacy.load('en_core_web_lg', disable = ['parser'])

In [53]:
nlp.pipe_names

['tagger', 'ner']

### Wordcount

In [54]:
Doc.set_extension('wordcount', default=0, force=True)

In [55]:
def wordcount(doc):
    '''gives an overall word count'''
    wordcount = 0
    
    for token in doc:
        if token.is_alpha:
            wordcount += 1
            
    doc._.wordcount = wordcount
    
    print(f'{wordcount} overall words')
         
    return doc

In [56]:
nlp.add_pipe(wordcount)
nlp.pipe_names

['tagger', 'ner', 'wordcount']

### Filter

In [57]:
Token.set_extension('is_excluded', default=False, force=True)

In [58]:
def filter_tokens(doc):
    '''filters all tokens'''           
    
    for token in doc:
        # filter stopwords
        if not token.is_alpha or token.is_stop:
            token._.is_excluded = True
        # filter part-of-speech
        elif token.pos_ not in ['NOUN', 'VERB', 'ADJ', 'ADV']: # ADV?
            token._.is_excluded = True
        # filter entities
        elif token.ent_type != 0:
                token._.is_excluded = True
         
    return doc

In [59]:
def count_words(doc):
    c = 0
    
    for token in doc:
        if not token._.is_excluded:
            c += 1
    print(f'{c} words included')
    
    return doc

In [60]:
nlp.add_pipe(filter_tokens)
nlp.add_pipe(count_words)
nlp.pipe_names

['tagger', 'ner', 'wordcount', 'filter_tokens', 'count_words']

### Eliminating Duplicates

In [61]:
Token.set_extension('appearance', default=np.nan, force=True)

In [62]:
def elim_dup(doc):
    '''eliminates all duplicates and counts the appearance of the included words'''
    already_appeared = {}
    
    for token in doc:
        if not token._.is_excluded:
            if token.lemma_ in already_appeared.keys():
                already_appeared[token.lemma_] += 1
                token._.is_excluded = True
            else: 
                already_appeared[token.lemma_] = 1
    
    for token in doc:
        if not token._.is_excluded:
            token._.appearance = already_appeared[token.lemma_]
    
    
    
    return doc

In [63]:
def count_voc(doc):
    c = 0
    
    for token in doc:
        if not token._.is_excluded:
            c += 1
    print(f'{c} words without duplicate included')
    
    return doc

In [64]:
nlp.add_pipe(elim_dup)
nlp.add_pipe(count_voc)
nlp.pipe_names

['tagger',
 'ner',
 'wordcount',
 'filter_tokens',
 'count_words',
 'elim_dup',
 'count_voc']

### Relative Freqency

In [65]:
Token.set_extension('relativ_freq', default=np.nan, force=True)

In [66]:
def calculate_relativ_freq(doc):
    '''calculating the relativ frequency of a included word'''
    
    maw = 20 # maximal appearance weight
    
    for token in doc:
        if not token._.is_excluded:
            text_freq = min(maw, token._.appearance) / doc._.wordcount
            overall_freq = word_frequency(token.lemma_, 'en')
            if overall_freq != 0:
                token._.relativ_freq = text_freq**2 / overall_freq
        
    return doc

In [67]:
nlp.add_pipe(calculate_relativ_freq)
nlp.pipe_names

['tagger',
 'ner',
 'wordcount',
 'filter_tokens',
 'count_words',
 'elim_dup',
 'count_voc',
 'calculate_relativ_freq']

### Rating Difficulty

In [68]:
Token.set_extension('difficulty', default=0, force=True)

In [69]:
def get_difficulty(doc):
    
    for token in doc:
        if not token._.is_excluded:
            lemma = token.lemma_
            readability = zipf_frequency(lemma, 'en') # score of 1-8
            syl = syllable_counter(lemma)
            if syl > 1 and readability != 0 :
                readability -= 0 #(syl - 2)/2
            token._.difficulty = round(8 - readability, 2)
            
    return doc

In [70]:
nlp.add_pipe(get_difficulty)
nlp.pipe_names

['tagger',
 'ner',
 'wordcount',
 'filter_tokens',
 'count_words',
 'elim_dup',
 'count_voc',
 'calculate_relativ_freq',
 'get_difficulty']

### Word Clusters

In [71]:
import collections

In [72]:
'''Token.set_extension('cluster', default=np.nan, force=True)
Doc.set_extension('cluster_sizes', default=0, force=True)'''

"Token.set_extension('cluster', default=np.nan, force=True)\nDoc.set_extension('cluster_sizes', default=0, force=True)"

In [73]:
def cluster_words(doc):
    
    X = [tok.vector for tok in doc if not tok._.is_excluded]
    dist = nltk.cluster.util.cosine_distance
    k = int(round(np.log(doc._.wordcount), 0 )) #prototype
    print(k)
    
    kclusterer = nltk.cluster.KMeansClusterer(k, distance=dist)
    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)
    clusters = iter(assigned_clusters)
    
    for token in doc:
        if not token._.is_excluded:
            token._.cluster = next(clusters)
    
    doc._.cluster_sizes = collections.Counter(assigned_clusters)
    
    return doc

In [74]:
def exclude_smallest_clusters(doc):
    
    cluster_count = len(doc._.cluster_sizes)
    smallest_clusters = []
    for i in range(int(cluster_count * 2/3), cluster_count):
        c = doc._.cluster_sizes.most_common()[i][0]
        smallest_clusters.append(c)
        
    print(smallest_clusters)
        
    for token in doc:
        if not token._.is_excluded:
            if token._.cluster in smallest_clusters:
                token._.is_excluded = True
    
    return doc

In [75]:
'''nlp.add_pipe(cluster_words)
nlp.add_pipe(exclude_smallest_clusters)
nlp.pipe_names'''

'nlp.add_pipe(cluster_words)\nnlp.add_pipe(exclude_smallest_clusters)\nnlp.pipe_names'

### Keywords Extraction

In [76]:
Doc.set_extension('keywords', default=[], force=True)
Token.set_extension('is_keyword', default=False, force=True)
Token.set_extension('keyword_score', default=-20, force=True)

In [77]:
def check_keyphrases(doc):
    
    kw = keywords(doc.text, split=True)
    
    keywords_token = []
    already_in_list = []
    for token in doc:
        if token.text in kw:
            token._.is_keyword = True
            if token.text not in already_in_list:
                keywords_token.append(token)
                already_in_list.append(token.text)
    doc._.keywords = keywords_token
    
    print(f'{len(keywords_token)} keywords found')
    
    for token in doc:
        for kw in keywords_token:
            kw_score = token.similarity(kw)
            if kw_score > token._.keyword_score:
                token._.keyword_score =  kw_score
    
    return doc

In [78]:
nlp.add_pipe(check_keyphrases)
nlp.pipe_names

['tagger',
 'ner',
 'wordcount',
 'filter_tokens',
 'count_words',
 'elim_dup',
 'count_voc',
 'calculate_relativ_freq',
 'get_difficulty',
 'check_keyphrases']

# Test Processing

### Test scentence

In [238]:
doc = nlp("Hi I'm a little little boy. Get me a piece of cake or I'll killed your mother. I am the mother of your mother")

24 overall words
9 words included
6 words without duplicate included
Keywords: 0


In [208]:
doc._.keywords

[]

In [69]:
# doc._.cluster_sizes

In [70]:
data = []
for token in doc:
    if not token._.is_excluded:
        data.append((token, token.lemma_, token._.appearance, token._.difficulty, token._.relativ_freq, token._.cluster))
df = pd.DataFrame(data, columns=['token', 'lemma', 'appearance', 'difficulty', 'relativ freqency', 'cluster'])
df.head(100)

AttributeError: [E046] Can't retrieve unregistered extension attribute 'cluster'. Did you forget to call the `set_extension` method?

In [203]:
df.sort_values(by=['difficulty'], ascending=False)

,token,lemma,appearance,difficulty,relativ freqency,cluster
3,cake,cake,1,3.57,64.539447,0
2,piece,piece,1,3.03,18.607836,0
1,boy,boy,1,2.80,10.988045,1
4,mother,mother,3,2.72,81.806283,1
0,little,little,2,2.25,12.356663,1


In [204]:
df.sort_values(by=['relativ freqency'], ascending=False)

,token,lemma,appearance,difficulty,relativ freqency,cluster
4,mother,mother,3,2.72,81.806283,1
3,cake,cake,1,3.57,64.539447,0
2,piece,piece,1,3.03,18.607836,0
0,little,little,2,2.25,12.356663,1
1,boy,boy,1,2.80,10.988045,1


________________________________

### Kafka

In [240]:
with open(text_file, "r") as f:
    doc = nlp(f.read())

25062 overall words
8460 words included
2047 words without duplicate included
Keywords: 440


In [241]:
# doc._.keywords

In [39]:
doc._.cluster_sizes

Counter({6: 255,
         0: 225,
         7: 340,
         4: 245,
         8: 162,
         1: 268,
         2: 158,
         5: 120,
         3: 176,
         9: 73})

In [267]:
data = []
for token in doc:
    if not token._.is_excluded:
        data.append((token, token.lemma_, token._.appearance, token._.difficulty, token._.relativ_freq, token._.cluster))
df = pd.DataFrame(data, columns=['token', 'lemma', 'appearance', 'difficulty', 'relativ freqency', 'cluster'])
df.head(100)

,token,lemma,appearance,difficulty,relativ freqency,cluster
0,Translated,translate,3,4.03,0.001536,9
1,use,use,21,2.17,0.000942,9
2,cost,cost,6,2.79,0.000354,7
3,restrictions,restriction,2,4.18,0.000963,7
4,whatsoever,whatsoever,3,4.01,0.001467,7
...,...,...,...,...,...,...
95,worries,worry,6,3.16,0.000828,1
96,making,make,14,1.95,0.000279,1
97,connections,connection,2,3.23,0.000108,9
98,bad,bad,4,2.47,0.000075,1


In [268]:
df.sort_values(by=['difficulty'], ascending=False).head(20)

,token,lemma,appearance,difficulty,relativ freqency,cluster
302,swinging,swinge,1,8.00,NaN,6
827,saddened,sadden,1,5.97,0.014879,4
858,assailed,assail,1,5.91,0.012944,4
203,hearer,hearer,1,5.90,0.012636,4
1146,rumination,rumination,1,5.82,0.010544,4
735,incarcerated,incarcerate,1,5.81,0.010272,4
846,perversity,perversity,1,5.76,0.009150,4
711,construed,construe,1,5.76,0.009150,7
582,unthinking,unthinking,1,5.74,0.008748,4
923,protrusions,protrusion,1,5.68,0.007618,6


In [269]:
df.sort_values(by=['relativ freqency'], ascending=False).head(30)

,token,lemma,appearance,difficulty,relativ freqency,cluster
1208,distribute,distribute,17,4.21,0.074573,7
461,locksmith,locksmith,4,5.35,0.056988,7
1297,donations,donation,16,4.03,0.043684,7
780,violin,violin,12,4.28,0.043669,6
10,COPYRIGHTED,copyright,21,3.76,0.036600,9
656,couch,couch,17,3.89,0.035668,6
1214,trademark,trademark,11,4.14,0.026608,7
777,clothes,clothe,3,5.23,0.024327,6
554,protruding,protrude,2,5.57,0.023674,6
1238,refund,refund,9,4.22,0.021386,7


In [217]:
df.sort_values(by=['cluster'], ascending=True).head(50)

,token,lemma,appearance,difficulty,relativ freqency,cluster
0,Translated,translate,3,4.03,0.001536,0
941,events,event,1,2.85,0.000011,0
959,normally,normally,3,3.40,0.000360,0
966,write,write,8,2.99,0.000999,0
967,reading,reading,1,2.92,0.000013,0
968,recent,recent,1,2.97,0.000015,0
972,wealth,wealth,1,3.48,0.000048,0
983,arrange,arrange,3,3.99,0.001405,0
990,ease,ease,1,3.66,0.000073,0
994,frequently,frequently,2,3.43,0.000171,0


### Wordclusters

In [242]:
doc = nlp("Let's take a ride in my new car. I will drive you home in my car. I love cars, which drive very fast and loud. Trucks and vans are the best. You can also cook in the back of my car. Food, meals and drinks are very tasty. My mother is all in to cooking good meals, foods and healthy dishes. This keeps me healthy while. I love the pasta and pies she bakes")

74 overall words
34 words included
24 words without duplicate included
Keywords: 3


In [148]:
doc[8]._.is_keyword

True

In [262]:
words = [[token.lemma_ for token in doc if not token._.is_excluded]]
print(words)

[['car', 'drive', 'fast', 'truck', 'van', 'good', 'cook', 'meal', 'food', 'healthy', 'dish', 'pasta', 'pie', 'bake']]


In [263]:
data = []
for token in doc:
    if not token._.is_excluded:
        data.append((token, token.lemma_, token._.appearance, token._.difficulty, token._.relativ_freq, token._.cluster))
df = pd.DataFrame(data, columns=['token', 'lemma', 'appearance', 'difficulty', 'relativ freqency', 'cluster'])
df.head(100)

,token,lemma,appearance,difficulty,relativ freqency,cluster
0,cars,car,1,2.58,2.261916,3
1,drive,drive,1,2.91,4.836455,3
2,fast,fast,1,2.91,4.836455,3
3,Trucks,truck,1,3.45,16.757296,3
4,vans,van,1,3.20,9.427639,3
5,best,good,2,1.88,1.802679,3
6,cooking,cook,1,3.37,13.931710,1
7,meals,meal,1,3.53,20.165559,1
8,foods,food,1,2.58,2.261916,1
9,healthy,healthy,2,3.22,39.461625,1


In [264]:
doc._.cluster_sizes

Counter({0: 3, 3: 6, 1: 8, 2: 1})

### Wikipedia

In [44]:
text = '''Deep learning (also known as deep structured learning) is part of a broader family 
of machine learning methods based on artificial neural networks with representation 
        learning. Learning can be supervised, semi-supervised or unsupervised.[1][2][3]Deep-learning 
        architectures such as deep neural networks, deep belief networks, recurrent neural networks 
        and convolutional neural networks have been applied to fields including computer vision, 
        machine vision, speech recognition, natural language processing, audio recognition, social 
        network filtering, machine translation, bioinformatics, drug design, medical image analysis, 
        material inspection and board game programs, where they have produced results comparable 
        to and in some cases surpassing human expert performance.[4][5][6]Artificial neural networks 
        (ANNs) were inspired by information processing and distributed communication nodes in biological 
        systems. ANNs have various differences from biological brains. Specifically, neural networks tend 
        to be static and symbolic, while the biological brain of most living organisms is dynamic (plastic) 
        and analogue.[7][8][9]The adjective "deep" in deep learning refers to the use of multiple layers in 
        the network. Early work showed that a linear perceptron cannot be a universal classifier, and then 
        that a network with a nonpolynomial activation function with one hidden layer of unbounded width 
        can on the other hand so be. Deep learning is a modern variation which is concerned with an unbounded
        number of layers of bounded size, which permits practical application and optimized implementation, 
        while retaining theoretical universality under mild conditions. In deep learning the layers are 
        also permitted to be heterogeneous and to deviate widely from biologically informed connectionist 
        models, for the sake of efficiency, trainability and understandability, whence the "structured" part.'''

In [79]:
doc = nlp(text)

257 overall words
156 words included
114 words without duplicate included
17 keywords found


<ipython-input-77-6e5208dea2dd>:19: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  kw_score = token.similarity(kw)


In [84]:
for token in doc:
    if not token._.is_excluded:
        print(token._.keyword_score)

0.3938454
0.5744245
0.33924755
0.47624484
0.43483993
0.3270241
1.0
0.52090365
0.4847936
0.46523783
0.49933636
0.8762495
0.44071323
0.2955146
0.25625694
0.2955146
0.5744245
0.40140018
0.5100599
0.31690955
0.22550173
0.44786528
0.38716203
0.4775482
0.53911096
0.41549888
0.4988885
1.0
0.5007288
0.4610773
1.0
0.40415663
0.5314103
0.37051588
0.40864474
0.38324967
0.48531094
0.45266506
0.38082403
0.50242513
1.0
0.40681517
0.31428733
1.0
0.43199307
0.4897565
1.0
0.4878846
0.36730403
1.0
0.59545803
0.45384392
0.36053327
1.0
0.46185136
1.0
0.43217084
1.0
0.52928066
0.4549633
0.3777898
0.43885174
0.35806483
0.34795207
0.43768883
0.35460824
0.6406614
0.4451714
0.47006604
0.26633573
0.36964554
0.47620687
0.45806268
1.0
0.3019013
0.4692436
0.44344023
0.34928936
1.0
0.31417662
0.0
1.0
0.48357797
0.31554163
1.0
0.40788516
0.48470917
0.5270915
0.38272974
0.47252935
0.50294
0.49841097
0.34320146
0.44353783
1.0
0.5282285
0.4437383
0.45886028
0.42962694
0.59137577
0.49776393
0.23172696
0.38636324
0.43876